In [1]:
import os
import requests
import pandas as pd
import numpy as np
#from annotationframeworkclient import imagery
#from annotationframeworkclient.jsonservice import JSONService
#from annotationframeworkclient import imagery
import cloudvolume
from caveclient import CAVEclient
import nglui
from statistics import mean
import matplotlib.pyplot as plt

client = CAVEclient('minnie65_phase3_v1')
client.info.get_datastack_info()

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
# MAIN
# combine all analysed synapses in the manuscript

# ET neurons from the column
synapse_table_column = pd.read_feather("ET_Column_synapse_table.feather")
synapse_table_column = synapse_table_column[['synapse_id','pre_nucleus_id']]

print('number of individual synapses used from ET column neurons:',len(synapse_table_column))

# ET neurons that were fully extended
synapse_table_extended = pd.read_feather("ET_extended_synapse_table.feather")
synapse_table_extended = synapse_table_extended[['synapse_id','pre_nucleus_id']]
print('number of individual synapses used from ET fully extended neurons:',len(synapse_table_extended))

# Merge tables
synapse_table_concat = pd.concat([synapse_table_column, synapse_table_extended], axis=0)
synapse_table_concat = synapse_table_concat.drop_duplicates(subset='synapse_id', keep='first')
print('total number of individual synapses:', len(synapse_table_concat))
print('number of individual presynaptic ET neurons:', len(synapse_table_concat.pre_nucleus_id.unique()))




number of individual synapses used from ET column neurons: 3736
number of individual synapses used from ET fully extended neurons: 4162
total number of individual synapses: 7506
number of individual presynaptic ET neurons: 47


In [ ]:
# MAIN
#'y' location of cortical surface
surface_y_column =[84534, 85689, 86053, 87800, 89421, 90105, 82884, 81677, 86242]
average_surface_location = mean(surface_y_column)

In [ ]:
#Number of synapses per ET neuron

# Loaad ET neurons that were fully extended
synapse_table = pd.read_feather("ET_extended_synapse_table.feather")


client = CAVEclient('minnie65_phase3_v1')
client.info.get_datastack_info()

# setup an empty list
ds=[]

#Number of inputs and outputs for each PT neuron
ET_IDs = synapse_table.pre_pt_root_id.unique()

for ET_id in synapse_table.pre_pt_root_id.unique():

        print(ET_id)
        
        d={
                        'ET_soma_ID': ET_id,
                        'number_output_synapses': len(synapse_table[synapse_table['pre_pt_root_id'] == ET_id]),
                        'number_input_synapses': len(client.materialize.synapse_query(post_ids=ET_id)),
        }
        
        #append entry
        ds.append(d)

ET_io = pd.DataFrame(ds)
ET_io   

864691135730543289
864691136903585714
864691135082322679
864691137020299886
864691135397023777
864691135162983725
864691135447395156
864691135293076662
864691135492596063
864691136137767293
864691136009704622
864691135572084717


In [ ]:
np.mean(pt_io)

In [ ]:
np.std(pt_io)

In [ ]:
np.min(pt_io)

# FIGURE 1

# FIGURE 2

In [ ]:
# ET neurons from the column
synapse_table_column = pd.read_feather("ET_Column_synapse_table.feather")

#Number of synapses with inhibitory targets
print('number of synapses with inhibitpry targets:', 
      len(synapse_table_column.query("consensus_class == 'inhibitory'")))

#Number of synapses with excitatory targets
print('number of synapses with excitatory targets:', 
      len(synapse_table_column.query("consensus_class == 'excitatory'")))


In [ ]:
# DEBUG CELL
ii = 864691135270770597
synapse_table[synapse_table['post_pt_root_id'] == ii]#.manual_class.unique()
#len(synapse_table[synapse_table['post_pt_root_id'] == ii]) 

In [ ]:
# DEBUG CELL

#len(synapse_table[(synapse_table['check_class_from_subclass'] != 'OK')])# & (pd.notna(synapse_table['check_class_from_subclass']))]) # (synapse_table['check_class_from_subclass'] != None)])
synapse_table[(synapse_table['check_class_from_subclass'] == '5P-NP')]# & (pd.notna(synapse_table['check_class_from_subclass']))]) # (synapse_table['check_class_from_subclass'] != None)])

#synapse_table[(synapse_table['id'] == 225174082)]
#synapse_table[(synapse_table['check_class_from_subclass'] != 'OK') & (pd.notna(synapse_table['check_class_from_subclass']))] # (synapse_table['check_class_from_subclass'] != None)]

In [ ]:
#CREATE NEUROGLANCER LINK


#manual_check = synapse_table[(synapse_table['synapse_id']== 183357055)] #.drop_duplicates(subset='post_pt_root_id')
#manual_check = synapse_table[(synapse_table['synapse_id']== 169481027)].drop_duplicates(subset='post_pt_root_id')
#manual_check = synapse_table[(synapse_table['ei_baylor'] == 'inhibitory') 
#              & (synapse_table['ei_aibs'] == 'excitatory') & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

manual_check = synapse_table[(synapse_table['consensus_subclass'] == '5P-ET')]#.drop_duplicates(subset='post_pt_root_id')
#manual_check = synapse_table_extended[pd.isna(synapse_table_extended.num_soma)]



from nglui import statebuilder

img, seg = statebuilder.from_client(client)

pt_map = statebuilder.PointMapper('post_pt_position', linked_segmentation_column='post_pt_root_id')
anno = statebuilder.AnnotationLayerConfig('post_pt_position', mapping_rules=pt_map, linked_segmentation_layer=seg.name,
                                          tags=['excitatory','inhibitory', 'MC', '6P', 'BC', '5P-NP', '5P-IT', '5P-PT'])
sb = statebuilder.StateBuilder([img, seg, anno], client=client)

#here is where you add the dataframe
sb.render_state(manual_check[['post_pt_root_id','post_pt_position']], return_as='html')

#[id, x,y,z]

In [ ]:
len(manual_check)

In [ ]:
synapse_table_extended.head()

In [ ]:
#OPTIONAL

#QC - CHECK DISAGREEMENT BETWEEN AUTOMATED AND MANUAL SUBCLASS LABELS
#Iterate  over different combinations
synapse_table[(synapse_table['manual_class'] == 'inhibitory') 
              & (synapse_table['ei_aibs'] == 'inhibitory')
              & (synapse_table['ei_aibs'] == 'inhibitory')]


In [ ]:
# MAIN
#INTEGRATE CLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['manual_eiaibs_class'] = synapse_table['manual_class']

#When there isn't manual label add aibs label

def integrate_class(row):
    if row['manual_eiaibs_class'] == None:
          return row['ei_aibs']
    
    else:
          return row['manual_eiaibs_class']  

synapse_table['manual_eiaibs_class'] = synapse_table.apply(integrate_class, axis=1)  

In [ ]:
# MAIN
#INTEGRATE SUBCLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['manual_eiaibs_subclass'] = synapse_table['manual_subclass']

#When there isn't manual label add aibs_v2 label

def integrate_subclass(row):
    if row['manual_eiaibs_subclass'] == None:
          return row['aibs_auto_subclass']
    
    else:
          return row['manual_eiaibs_subclass']  

synapse_table['manual_eiaibs_subclass'] = synapse_table.apply(integrate_subclass, axis=1)  

In [ ]:
# MAIN
#QC - CHECK IF THE INTEGRATED CLASS AND SUBCLASS ARE CONSISTENT


#create new column where class is calculated from subclass
def create_class_from_subclass(row):
    if row['manual_eiaibs_subclass'] == '5P-NP':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '5P-PT':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '5P-IT':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '4P':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '6P':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '6P-IT':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '6P-CT':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == '23P':
          return 'excitatory'
    if row['manual_eiaibs_subclass'] == 'BC':
          return 'inhibitory'
    if row['manual_eiaibs_subclass'] == 'MC':
          return 'inhibitory'
    if row['manual_eiaibs_subclass'] == 'NGC':
          return 'inhibitory'
    if row['manual_eiaibs_subclass'] == 'BPC':
          return 'inhibitory'
    else:
          return row['manual_eiaibs_subclass']

synapse_table['class_from_subclass'] = synapse_table.apply(create_class_from_subclass, axis=1)


def check_class_from_subclass(row):
   
    if row['manual_eiaibs_class'] == row['class_from_subclass']:
          return 'OK'
   
    else:
          return row['manual_eiaibs_subclass']  

synapse_table['check_class_from_subclass'] = synapse_table.apply(check_class_from_subclass, axis=1)

synapse_table.check_class_from_subclass.unique()


In [ ]:
#len(synapse_table[(synapse_table['check_class_from_subclass'] != 'OK')])# & (pd.notna(synapse_table['check_class_from_subclass']))]) # (synapse_table['check_class_from_subclass'] != None)])
synapse_table[(synapse_table['check_class_from_subclass'] == 'MC')]# & (pd.notna(synapse_table['check_class_from_subclass']))]) # (synapse_table['check_class_from_subclass'] != None)])
#synapse_table[(synapse_table['manual_eiaibs_subclass'] == 'inhibitory')]# & (pd.notna(synapse_table['check_class_from_subclass']))]) # (synapse_table['check_class_from_subclass'] != None)])

#synapse_table[(synapse_table['id'] == 225174082)]
#synapse_table[(synapse_table['check_class_from_subclass'] != 'OK') & (pd.notna(synapse_table['check_class_from_subclass']))] # (synapse_table['check_class_from_subclass'] != None)]

In [ ]:
# MAIN

synapse_table.loc[1958,'manual_eiaibs_subclass']  = None
synapse_table.loc[1958]

In [ ]:
#QC - CHECK DISAGREEMENT BETWEEN AUTOMATED (AIBS AND BAYLOR) AND MANUAL LABELS
#Iterate  over different combinations

def check_subclass_consensus(row):
   
    if row['manual_subclass'] == row['aibs_auto_subclass']:
          return 'OK'
   
    else:
          return row['manual_subclass']  

synapse_table['subclass_consensus'] = synapse_table.apply(check_subclass_consensus, axis=1)




In [ ]:
synapse_table[(synapse_table['manual_class'] == 'inhibitory') 
              & (synapse_table['ei_baylor'] == 'inhibitory')
              & (synapse_table['subclass_consensus'] == 'MC')
              & pd.notna(synapse_table['manual_subclass'])].drop_duplicates(subset=['post_pt_root_id'])



In [ ]:
synapse_table.head()

In [ ]:
synapse_table.pre_pt_root_id.unique()

In [ ]:
# MAIN

#SAVE DATAFRAME
#synapse_table.to_csv("syn_df_updated_feb4__feb5.csv")
synapse_table.reset_index(drop=True).to_feather("syn_analysis_column_MANUSCRIPT.feather")

In [ ]:
#COMPARE CLASS LABELS

#Neurons for which the baylor and aibs classifications desagree and there is a classification for Baylor
#
#class_comparison = synapse_table[(synapse_table['ei_baylor'] != synapse_table['ei_aibs'])  &
#                                    pd.notna(synapse_table['ei_baylor'])]


#Neurons for which the baylor and aibs classifications desagree and there is a classifaction for AIBS
#and there is no manual classification
#
class_comparison = synapse_table[(synapse_table['is_axon'] == True) & (synapse_table['ei_baylor'] != synapse_table['ei_aibs_v2'])  &
                                    pd.notna(synapse_table['ei_aibs_v2']) & pd.isnull(synapse_table['manual_class'])]
                                    

#Neurons for which the baylor has Class label and aibs does not
#
#class_comparison = synapse_table[pd.notna(synapse_table['ei_baylor']) & pd.isnull(synapse_table['ei_aibs'])]


#Other
#class_comparison = synapse_table[(synapse_table['ei_baylor'] != synapse_table['ei_aibs'])]

#class_comparison = synapse_table[(synapse_table['ei_baylor'] != synapse_table['ei_aibs'])  &
#                                    (pd.notna(synapse_table['ei_aibs']) & pd.isnull(synapse_table['ei_baylor']))]



In [ ]:
len(synapse_table.post_pt_root_id.unique())

In [ ]:
#ANALYSIS - GENERAL

#NUMBER OF SYNAPSES

synapse_numbers = { 'total_synapses' : len(synapse_table),
                    
                    'synapses_with_class_labels' : len(synapse_table[pd.notna(synapse_table['manual_eiaibs_class'])]),
                    'synapses_with_class_labels_manual' : len(synapse_table[pd.notna(synapse_table['manual_class'])]),
                    'synapses_without_class_labels' : len(synapse_table[pd.isna(synapse_table['manual_eiaibs_class'])]),
                   
                   
                    'synapses_with_subclass_labels' : len(synapse_table[pd.notna(synapse_table['manual_eiaibs_subclass'])]),
                    'synapses with_subclass_labels_manual' : len(synapse_table[pd.notna(synapse_table['manual_subclass'])]),
                    'synapses_without_subclass_labels' : len(synapse_table[pd.isna(synapse_table['manual_eiaibs_subclass'])]),
                   
                    'synapses_with_orphans' : len(synapse_table[synapse_table['num_soma'] == 0]),
                    'synapses_with_orphans_with_class_labels' : len(synapse_table[(synapse_table['num_soma'] == 0) &
                                                                    pd.notna(synapse_table['manual_eiaibs_class'])]),
                    'synapses_with_orphans_without_class_labels' : len(synapse_table[(synapse_table['num_soma'] == 0) &
                                                                    pd.isna(synapse_table['manual_eiaibs_class'])]),
                    'synapses_with_orphans_without_class_labels_spines': [],
                    'synapses_with_orphans_without_class_labels_dendrites': [],
                    'synapses_with_orphans_with_subclass_labels' : len(synapse_table[(synapse_table['num_soma'] == 0) &
                                                                    pd.notna(synapse_table['manual_eiaibs_subclass'])]),
                    'synapses_with_orphans_without_subclass_labels' : len(synapse_table[(synapse_table['num_soma'] == 0) &
                                                                    pd.isna(synapse_table['manual_eiaibs_subclass'])]),

                                   
                    'synapses_with_multisoma' : len(synapse_table[synapse_table['num_soma'] > 1]),
                    'synapses_with_multisoma_with_class_labels' : len(synapse_table[(synapse_table['num_soma'] > 1) &
                                                                    pd.notna(synapse_table['manual_eiaibs_class'])]),
                    'synapses_with_multisoma_without_class_labels' : len(synapse_table[(synapse_table['num_soma'] > 1) &
                                                                    pd.isna(synapse_table['manual_eiaibs_class'])]),
}


#NUMBER OF SOMATA

connection_numbers = {
                    'total_connections' : len(synapse_table[synapse_table['num_soma'] == 1].post_pt_root_id.unique()),
                   
                    'with_class_labels' : len(synapse_table[(pd.notna(synapse_table['manual_eiaibs_class'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
                    'with_class_labels_manual' : len(synapse_table[(pd.notna(synapse_table['manual_class'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
                    'without_class_labels' : len(synapse_table[(pd.isna(synapse_table['manual_eiaibs_class'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
                
    
                   
                    'with_subclass_labels' : len(synapse_table[(pd.notna(synapse_table['manual_eiaibs_subclass'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
                    'with_subclass_labels_manual' : len(synapse_table[(pd.notna(synapse_table['manual_subclass'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
                    'without_subclass_labels' : len(synapse_table[(pd.isna(synapse_table['manual_eiaibs_subclass'])) &
                                                             (synapse_table['num_soma'] == 1)].post_pt_root_id.unique()),
   
}

connection_numbers, synapse_numbers

In [ ]:
#ANALYSIS - FIGURE 2
#Number of synapses per ET neuron

client = CAVEclient('minnie65_phase3_v1')
client.info.get_datastack_info()

# setup an empty list
ds=[]

#Number of inputs and outputs for each PT neuron
pt_IDs = synapse_table.pre_pt_root_id.unique()

for pt_id in synapse_table.pre_pt_root_id.unique():

        print(pt_id)
        
        d={
                        'pt_soma_ID': pt_id,
                        'number_output_synapses': len(synapse_table[synapse_table['pre_pt_root_id'] == pt_id]),
                        'number_input_synapses': len(client.materialize.synapse_query(post_ids=pt_id)),
        }
        
        #append entry
        ds.append(d)

pt_io = pd.DataFrame(ds)
pt_io

In [ ]:
synapse_table[(synapse_table['pre_pt_root_id'] == 864691135367970809) &
                                     (synapse_table['manual_eiaibs_subclass'] == '4P')])





In [ ]:
(client.materialize.query_table('nucleus_detection_v0', 
                               filter_equal_dict={'id':)['pt_position'].apply(lambda x: x[0]) - average_surface_location)*0.004

In [ ]:
anno_df['point'].apply(lambda x: x[0])

In [ ]:
synapse_table.motif_group.unique()


In [ ]:
len(synapse_table[ (synapse_table['pre_pt_root_id'] == 864691135660737776)]['post_pt_root_id'].unique())

In [ ]:
(client.materialize.query_table('nucleus_detection_v0', 
                               filter_equal_dict={'id':pre_soma_ID})['pt_position'].apply(lambda x: x[0]).iloc[0])

In [ ]:
synapse_table = synapse_table.rename(columns={"pre_nucleus_id2": "pre_nucleus_id"})


In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(synapse_table[synapse_table['consensus_subclass'] == '5P-ET']['size'], 
                     synapse_table[synapse_table['consensus_subclass'] == '5P-NP']['size'], method='asymptotic')
print(p)
